In [4]:


import keras

ModuleNotFoundError: No module named 'keras'

In [5]:
pip install Keras


     |████████████████████████████████| 378kB 4.6MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [24]:
from keras.models import Sequential


In [25]:
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, AvgPool2D, BatchNormalization, Reshape

In [26]:
from keras.preprocessing.image import ImageDataGenerator

In [27]:
from keras.callbacks import LearningRateScheduler

In [29]:
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** (x + epochs))
epochs = 15

In [30]:
import pickle

In [31]:
with open('ToySet1.pickle', 'rb') as f:
    data = pickle.load(f, encoding='latin1')

In [32]:

data['y_train'] = to_categorical(data['y_train'], num_classes=43)
data['y_validation'] = to_categorical(data['y_validation'], num_classes=43)

# Making channels come at the end
data['x_train'] = data['x_train'].transpose(0, 2, 3, 1)
data['x_validation'] = data['x_validation'].transpose(0, 2, 3, 1)
data['x_test'] = data['x_test'].transpose(0, 2, 3, 1)

# Showing loaded data from file
for i, j in data.items():
    if i == 'labels':
        print(i + ':', len(j))
    else: 
        print(i + ':', j.shape)

x_train: (540, 32, 32, 1)
y_train: (540, 43)
x_validation: (90, 32, 32, 1)
y_validation: (90, 43)
x_test: (180, 32, 32, 1)
y_test: (180,)
labels: 43


In [33]:
h = model.fit(data['x_train'][:10], data['y_train'][:10],
              batch_size=5, epochs = epochs,
              validation_data = (data['x_validation'], data['y_validation']),
              callbacks=[annealer], verbose=1)

Train on 10 samples, validate on 90 samples
Epoch 1/15
10/10 [==============================] - 0s 30ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 14.4705 - val_accuracy: 0.7111
Epoch 2/15
10/10 [==============================] - 0s 23ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 14.5726 - val_accuracy: 0.7222
Epoch 3/15
10/10 [==============================] - 0s 23ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 14.7983 - val_accuracy: 0.7222
Epoch 4/15
10/10 [==============================] - 0s 34ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 15.1811 - val_accuracy: 0.7333
Epoch 5/15
10/10 [==============================] - 0s 26ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 15.5940 - val_accuracy: 0.7333
Epoch 6/15
10/10 [==============================] - 1s 58ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 15.9647 - val_accuracy: 0.7444
Epoch 7/15
10/10 [==============================] - 0s 24ms/step - loss: 0.0000e